In [25]:

from anthropic import Anthropic
import os
from dotenv import load_dotenv
import prompts

load_dotenv()

anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

sys_prompt = """
Your job is to review the user's notes, the transcript of a meeting, to then summarize the meeting.

Review the topic and notes to understand the context of the meeting and topics discussed. 

The summary will be structured as a list of bullet points, with headings separating the topics.

Attempt to arrange the meeting in a chronological order. 

Your bullet points will include the user's notes as far as possible.

Prioritize dates, monies, names, and other important information.
"""

meeting_transcript = """
Me: Hello. Good afternoon.  
Them: Good afternoon. You okay?  
Me: How's it going? I'm doing good. I'm I'm doing well. Thank you. How are you?  
Them: I'm fine.  
Me: I'm fine. Thank you so much for taking my call.  
Them: You so much. Thank you for taking my call.  
Me: So may I have the new insured  
Them: So we're having a few issues with connecting to Teams this afternoon.  
Me: of it. Bye bye.  
Them: About yourself?  
Me: Yeah. Doing well, actually. Thank you for reaching out. It was a nice surprise. I had just posted the, Spritely AI  
Them: Mhmm.  
Me: Mhmm. And so, yeah, I guess that that was a good sign. Maybe it's a good omen that, you know, this is a good project. Oh, brilliant.  
Them: Oh, brilliant. So, yeah, just tell me more about yourself, then I'll go through the Eagle Labs. I'm sure  
Me: So, yeah, just tell me more about yourself, then I'll go through the equalabs. I'm sure yours is more interesting than what I've got to say. So, yeah, let me  
Them: yours is more interesting than what I've got to say. So, yeah, let me  
Me: let me  
Them: tell me more about yourself and the business.  
Me: Thank Sure. So,  
Them: Fees.  
Me: I I'm I'm I haven't been a, sort of a techie person  
Them: Mhmm.  
Me: developer too long, but I've always I think I've always been, like, very technical at heart.  
Them: Mhmm. Mhmm.  
Me: I recently gotten into software.  
Them: Mhmm.  
Me: So last year,  
Them: Mhmm.  
Me: And in February, I met a  
Them: Mhmm.  
Me: someone who was technical. And so I was in restructuring at the time.  
Them: Mhmm.  
Me: So he introduced me to large language models, you know, AI and stuff like that.  
Them: Mhmm.  
Me: So that was, like, the first introduction.  
Them: Mhmm.  
Me: And so in June, I decided I'm gonna I'm gonna leave.  
Them: Mhmm.  
Me: Leave my career and and pursue, tech. And and so I went into what what they call marker mode.  
Them: Hello? Mhmm.  
Me: I went free for just, abroad, like, just  
Them: Wow.  
Me: coding for, like, ungodly hours a day. And then so, focusing on Spritely. I the vision is that it's a desktop based, AI application.  
Them: Mhmm.  
Me: So it connects, you know, your favorite  
Them: Mhmm. Okay.  
Me: AI apps. So whether that's Okay. Or, like, Claude if you use the anthropic model,  
Them: Okay.  
Me: Okay. And then it connects, you know, the, like, the text to speech  
Them: Okay.  
Me: Okay. So that's the that's like the mouth. So that's that's when  
Them: Mhmm.  
Me: talk and then  
Them: Mhmm.  
Me: text as well when your words are transcribed into into text.  
Them: Oh, wow.  
Me: Okay. Hi. So if you hit the main panel, you have the confounder.  
Them: Oh, so are you the main founder, or do you have a cofounder?  
Me: Yeah. So it's a solo project at the moment. It's just me.  
Them: Oh, fantastic. Alright. And is is the business been registered on company files and everything?  
Me: And it's it's the business been registered on company files and everything? Not under Spritely. But I I I I do have a limited company that I could  
Them: Okay.  
Me: Okay. Hi, Craig.  
Them: Okay. So if you can put it under the chat, please, so that I'd be able to  
Me: Able to, go for it. So what would you say the move is?  
Them: go forward. So what would you say at a moment  
Me: Did you know anything about the equal out? So  
Them: you know anything about the Eagle Lamps or  
Me: Okay.  
Them: have you heard of us in any way at all?  
Me: Yeah. I did. So on the day My last day at work  
Them: Mhmm.  
Me: I actually went to a Barclays Eu Lab like,  
Them: Okay. Okay.  
Me: So you guys do those, and and that was after work.  
Them: Mhmm.  
Me: So I went there. Network and to meet people.  
Them: Mhmm.  
Me: Mhmm. Mhmm. Yeah. I mean, Eagle Labs looks very promising. It's  
Them: Mhmm.  
Me: it's very good network and I'd love to  
Them: Yeah.  
Me: hear from yourself.  
Them: That's  
Me: Definitely. Yes. Of course. So where are you based? Not too  
Them: Definitely. Yes. Of course. So where are you based in London? First of all,  
Me: first of all? West London. So Hillingdon. Oh, Hillingdon. Okay.  
Them: I'm here in London. Okay. So you're still in London. Fantastic.  
Me: Yeah. So  
Them: Yeah. So, lovely. Lovely to hear about the business. So,  
Me: Lovely. You want me to hear about the business? So I don't mean that you  
Them: at the moment, are you within with Razor? Are you what are you looking to do now  
Me: what would you say  
Them: at the moment as a founder? What would you say is the greatest challenge for you  
Me: the greatest Charlie? So at the moment, I'm purging a lot of focus on making this  
Them: Mhmm.  
Me: known, so sharing it amongst developer communities. So it's open source.  
Them: Mhmm. Yeah.  
Me: So a benefit of that is people will contribute, people will look at the code, I can get feedback. And so last night when I  
Them: Mhmm.  
Me: I was doing like a sort of a a mini update to the website and to the app, so I see that. I I I posted on  
Them: Yeah.  
Me: Reddit. So it's already got 6 stars.  
Them: Yeah.  
Me: On GitHub. So it's good kinda picking up that momentum. But  
Them: 1,  
Me: Yeah. The idea is is that is that I'll continue to make content and to continue to get feedback.  
Them: I think  
Me: Interesting. And  
Them: And just take  
Me: yes. I I think this will like, in terms of how monetization will work, it'll probably be it'll probably be some kind of freemium model. So some of the versions are  
Them: Mhmm.  
Me: the open source code. Mhmm. And then as time goes by, you're you're like a cat.  
Them: Mhmm. You've  
Me: Separate some of the paid features. Fantastic.  
Them: Fantastic.  
Me: So I'm just gonna share  
Them: So I'm just gonna share my screen with you.  
Me: Okay. So  
Them: So let me know when you can see the screen, please.  
Me: can see the screen. Yep. I can see that. Fantastic.  
Them: Fantastic. So  
Me: So Was that  
Them: Without worrying you today so much about the Huber Labs,  
Me: So, essentially, we have been we have got over this business back  
Them: so, essentially, we've been we are part of Barclays Business Banking.  
Me: in. We've been around since 2015, so we've got 52 notifications.  
Them: We've been around since 2015. So we've got 42 lab locations around the country.  
Me: Around the country. You've attended Monifa Defense, you know,  
Them: As you can see, we've we've attended one of our events. So, you know,  
Me: how we do by fence. So we  
Them: how we do our events. So we we've had, 30 over 38,000 worth of  
Me: 30 58,000 event attendees, and we supported them.  
Them: event attendees, and we supported over 17,000 businesses.  
Me: I'll go through with you. And last year,  
Them: We have growth programs, which I'll go through with you.  
Me: If it's 22 of those,  
Them: Last year, we delivered  
Me: Okay. And the answer I'm saying is, like, great.  
Them: 22 of those. And lifetime funding, we've raised 4 point  
Me: 81p.  
Them: 81,000,000,000  
Me: So they they all have  
Them: So the the eagle lab says we are evolving as we go.  
Me: As we go, So ecosystem  
Them: So as the ecosystem,  
Me: with a It's free.  
Them: what the proposition is, it's free  
Me: For any founder Oh, you know, at  
Them: for any founder or, you know, any start up that's looking to scale.  
Me: looking to scale. So we have a dedicated programs growth programs  
Them: So we have, dedicated programs, growth programs,  
Me: We have industry specialists that  
Them: We have industry specialists. And for example, we've got AI community.  
Me: And with that, this can actually AI breakfast event.  
Them: And with that, there's a monthly AI breakfast event.  
Me: Which is some Shortage. I'd like  
Them: Which is, Shoreditch, Rice, one of our partners,  
Me: The special  
Them: that specializes in AI. So we have breakfast for members to essentially connect  
Me: meet our shared best practices.  
Them: and meet up and share best practices, you know, network. We have that.  
Me: You know, network. We have that. We also have test and connectivity. So we have a platform called the demo directory. I'll show you later.  
Them: Also have investor connectivity. So we have a platform have a  
Me: But it's a free platform for founders and investors to connect.  
Them: for founders and investors to connect. It's free for all  
Me: It's free for all. So you get to pinch a video a video of yourself in your business.  
Them: so you get to pitch a video a video of yourself and your business.  
Me: And if it's something that any person is interested in, you can make  
Them: And if it's something that an investor's interested, then you connect  
Me: And, you know, you let them run the campaign. We also  
Them: and, you know, you let the magic happen. We also have events and work  
Me: have events and workshops, across the country and the  
Them: workshops, across the country and within our systems as well.  
Me: systems as well. Have something also called corporate connections.  
Them: We have something also called corporate connections.  
Me: At the corporate connections, you may have a corporation that have an issue.  
Them: How the corporate connections so you may have a corporation may have an issue.  
Me: Being a huge corporation, sir, and the bureaucracy around it.  
Them: Being as huge as corporations are and the bureaucracy around it.  
Me: A founder may have a solution. So what we do is we put it on the platform, what the  
Them: A founder may have a solution. So what we do is  
Me: the corporate will put the challenge on our platform.  
Them: we put it on a platform where the the corporate will put  
Me: And if a candidate's interested,  
Them: the challenge on our platform. And if a founder's interested,  
Me: they're already stopped.  
Them: will register  
Me: I already said that they're interested in the ones that you  
Them: they will register that they are interested, and then once again, you get  
Me: You get connected. And then you go. So we have the customer.  
Them: connected, and then you go. So we have ecosystem expertise, for example,  
Me: 8 Pardon? That one's really cool. I like how it sort of helps  
Them: k. Put it.  
Me: bridge bridge Definitely. Corporate to the Definitely.  
Them: Yeah.  
Me: Yes.  
Them: Definitely. Definitely.  
Me: And it's really interesting. I'll but now I'll show you, like, what the challenges look like.  
Them: Yeah. And it's really interesting. When I I also was on  
Me: And what the normalization support for the challenges to be resolved. So  
Them: you, like, what the challenges look like and what different organizations have put up.  
Me: we also have within that  
Them: For the challenges to be resolved. So we also have within our ecosystem  
Me: system depending on industry. For your repair and sourcing, for example.  
Them: depending what industry vertical you've your business for, for example,  
Me: Bandwidth and sustainability.  
Them: we have the AI. We have climate and  
Me: We have,  
Them: climate and sustainability. We have,  
Me: Health. Health for the new signs. So we'll get them written  
Them: health, health and life science.  
Me: We have so many different  
Them: So we have games and creatives,  
Me: That's goes, you know, to support families on this scale. When you  
Them: have so many different verticals, you know, to  
Me: also have a free membership mentor. It  
Them: support founders and businesses scale up. We also, as a free membership,  
Me: You can free room. What that works is we've got partners in account to figure out, and  
Them: mentor, you get free rewards. What that works is we've got partners with Accountancy Cloud,  
Me: Some people pause this. Hang on. It's support. Sound to speak his  
Them: Seed Legal, so different partners who are there to support  
Me: me. And  
Them: founders because we understand the challenges of being a founder  
Me: So we try it  
Them: it's very costly. You know? So we try and see ways that we  
Me: that we can support And  
Them: support founders within their journey. And we also have  
Me: Now this is from  
Them: a free online platform. Now this is funded by the government So we were awarded £12,000,000  
Me: It's  
Them: 2 years ago for our digital growth fund. And how that works is we deliver certain program under the government funded program. So  
Me: Let's Right. That's  
Them: with the academy, like I say, it may not be relevant to you, but, essentially, it's taking a a founder through a journey from ideation to exit.  
Me: k. Case  
Them: We've got different modules in place with experts  
Me: that is, you know,  
Them: case studies, you know, different founders, different challenges. Different industries, you know, themes and stuff like that. So  
Me: 028  
Them: that's really and truly for people that are really starting early on  
Me: So, like, giving you the kind of back of education and support that you may need. And  
Them: in a journey. So, like, giving you the kind of back of education as  
Me: kind of like startups already. We have growth we have accelerate  
Them: support that you may need, for established kind of, like, startups already, we have growth  
Me: programs. So  
Them: We have accelerator programs. So our  
Me: 12 week calls.  
Them: accelerator programs, one of our partners being Plexor,  
Me: For families to really help to protect the next  
Them: So they deliver that 12 week course for founders to really help them look  
Me: but that looks right. So  
Them: at the lens of scaling up a business, what that looks like. Within our lab location, so  
Me: Manchester. We've actually got a Barclays and Lab there. We've got post work  
Them: labs, places like Cambridge, Oxford, Manchester. We've actually got a Barclays lab there that we've got colleague  
Me: in there. Now when we come to London, a different model. So for myself, I'm based in South London.  
Them: working there. Now when you come to London, it's a different model.  
Me: How that works is we go into partnership with co working spaces.  
Them: So for myself, I'm based in South London in Wandsworth. How that works is we go into partnership with co working spaces.  
Me: Put myself there to support founders in business.  
Them: And how that works is I will  
Me: That need our support. And my colleague, you wanna  
Them: base myself there to support founders and businesses that need our support.  
Me: So  
Them: And my colleague in West London, we've got a space with  
Me: I just moved. And also I've been choosing before.  
Them: Huddl. So we're in Fulham, Hammersmith, and also  
Me: So we are  
Them: up in Cheswick. We've got we've got x and y. So we are  
Me: We have had support in terms of just some  
Them: around London, but the the model is slightly different saying that  
Me: know, businesses that we need our support. We also offer patron and coaching free.  
Them: we are there to support any founders or, you know, businesses that may need our support.  
Me: For anybody. Within their ecosystem admin support. So in part, the credit card that they have looked  
Them: We also offer mentoring and coaching free for anybody  
Me: were offered us. We  
Them: within our ecosystem that may support. So part of the growth fund that the governor  
Me: Business Coaches. Sweet. For startups or businesses, then it has to go  
Them: offered us, we deliver business coaching  
Me: So What  
Them: and mentoring for startups or businesses that need our support.  
Me: It's  
Them: So once when we go through it, I'll show you each  
Me: Good night. Just a question, please. You  
Them: Our partners does provide the mentors and coaches are we've got Cambria,  
Me: mentioned the, plexil. Yep. How do you spell that?  
Them: Yeah.  
Me: I just wanna have a look.  
Them: Yeah.  
Me: So p n  
Them: So is  
Me: e I a l. Okay. And they're one of the accelerator  
Them: p l  
Me: As you mentioned,  
Them: exal.  
Me: Yeah. Okay. Yeah. And and just on the lab locations, is there any sort of, like,  
Them: Yes.  
Me: subsidy for,  
Them: Yes.  
Me: joining the  
Them: Yeah. Mhmm.  
Me: working spaces, or is that just And you have to go through the normal sort of channels Yeah. So And if  
Them: Yeah. So we we're just a partner there. So the partner will have his own  
Me: You know, you can't discount the stuff like that would begin.  
Them: prices. I mean, from time to time, they may have enough room placed to say as an Eagle Lab  
Me: It's the Space they kinda dictate the price  
Them: member, you know, you get discounts or stuff like that. But because it's their  
Me: stuff like that. We're just they ask Come in  
Them: space, they kinda dictates the price and the  
Me: Okay. Sounds good. Thank you. Yeah. I know.  
Them: stuff like that. We're just there as an added value for members.  
Me: I mean, no. That's fine. That's fine. Feel free to stop if you need  
Them: Yeah. And I'm  
Me: So  
Them: no. That's fine. That's fine. So please feel free to stop if you need me to answer any questions.  
Me: once again,  
Them: Questions at all. So once again, there there's no harm in trying. Some there has been times when they may offer a discount for a period of time.  
Me: ends and a partner ends. What is it that then? What they charge you? So and  
Them: So it really and truly depends on the partner and what is it that they  
Me: so for the veteran and coaches, these are our list  
Them: what they're charging us are. And so for the management and coaching,  
Me: To enter price,  
Them: these are our partners that we've partnered with. So we've got Cambridge  
Me: You know, the coat base carbonate. Altogether. So many  
Them: Judge Business School, Capital Enterprise,  
Me: got many  
Them: you know, code base, carbon 13,  
Me: partners who  
Them: altogether, accelerate her. So many we've got many more.  
Me: Support Now  
Them: Partners who help us to support businesses. Now for businesses that have a bit further  
Me: We have We have think we're capable  
Them: down and they're looking to scale up, we have the scale up program, like I mentioned.  
Me: So we You could get  
Them: Also have something called GlobalConnect. So with GlobalConnect,  
Me: Target sector market overseas. We have a contacted a Silicon Valley  
Them: how that works is if a business is looking to  
Me: Space. So we can then  
Them: target certain market overseas, we have actually got contacts within a Silicon Valley  
Me: It's So  
Them: space, so we connect them. Now this program is run once a year,  
Me: wait to go outside the  
Them: So, hopefully, in the new year, we'll be opening up to any businesses that's looking  
Me: outside the country. And we have that the connection to support  
Them: to scale up and looking at ways to go outside of mark  
Me: The business as well.  
Them: outside the country. And we have that kind of connection to support  
Me: Say enough. And  
Them: the business as well. So  
Me: So we've got private  
Them: Anessa and when we when I was talking about  
Me: platform that I was talking to you about.  
Them: the demo directory, so we've got private  
Me: Free to join. We've got 400  
Them: investors, the platform that I was talking to you about, free to join.  
Me: investors on it We've got angels and VCs. 36, so thanks for those.  
Them: We've got over 400  
Me: And how about  
Them: investors on there. We've  
Me: is designed to open businesses and founders to connect directly.  
Them: got angels and VCs, so a mix of those. And how that work essentially  
Me: We don't take any path. Time. On this, we get to profile for the business with a video  
Them: is designed to help investors and founders to connect directly  
Me: Now each  
Them: We don't take any cut. Founders create  
Me: Great.  
Them: profile for their business with a video for the investors to access. Now based on ranges, investors will look and see what is it that they've got they appetite  
Me: Now the investment range is 120 k  
Them: for, and they'll they'll look and see if there's something they're interested in.  
Me: So to 25,000,000 and  
Them: Now investment ranges from 20 k  
Me: Trans rate 1,200,000. And  
Them: So to 25,000,000. And average  
Me: We've Oh, this  
Them: fund raise is 1,200,000.  
Me: which is the only help once again.  
Them: And we also have a demo days, a way for founders  
Me: I had a case So we get  
Them: and investors to meet face to face, which is normally held once a year.  
Me: So section, this detail can see how big it We can connect. So that's how  
Them: Head office. So we get a lot of people in a room  
Me: And  
Them: to essentially just meet up and see how big you know?  
Me: posted connections.  
Them: We can connect. So that's how that works.  
Me: I was looking to you about. Imagine to collaborate  
Them: And we also the court this is the corporate connections that I was talking to you about in regards  
Me: with, you know, institutions and  
Them: to emerging technologies, co collaborating with, you know,  
Me: A business Correct. We  
Them: And institutions who mainly support and to help, you know, the business  
Me: Alright. Stupid We have plugged  
Them: grow. So we have  
Me: Check. Digital Security Center.  
Them: our industry programs at the moment. We have AI and emerging tech,  
Me: K. An 80 in the life science and health tech.  
Them: We have climate and nature innovation, digital security, Fintech.  
Me: So please tell me  
Them: Games, and creatives in the life science and health tech. So  
Me: you sign the very form of the check,  
Them: these are the industries, and they all have their own kind of communities. And how the community works is, essentially, once you sign a virtual membership,  
Me: With And  
Them: I'll show you, you'll literally click on whatever community that is of interest to you. And then with that, you get the latest news, any events happening around.  
Me: Version 10.  
Them: Any updates around, you know, the AI? For example, if you're doing AI and emerging tech,  
Me: We have Is  
Them: and there's anything what's happening in AI? Was was the landscape, you know, What technology? So it's literally, essentially, We haven't insights about what's happening around, which I think sometimes  
Me: We have  
Them: is really, really beneficial for founders.  
Me: So we have  
Them: And, also, as a virtual member,  
Me: What And  
Them: we have member rewards. So these are the kind of partnerships we have.  
Me: And  
Them: Where you get discounts with regards to whatever your business may need, whether it's  
Me: we  
Them: account in the cloud. So you  
Me: so with that,  
Them: you know, see labels. So really and truly, it's all on their website, and it's free.  
Me: just And then  
Them: For all virtual members. So with that, I'm just gonna also  
Me: What's I we in a second.  
Them: show I'm gonna stop sharing now, and then I'll show you what  
Me: The And How about  
Them: the website  
Me: Do you know a bit  
Them: Bear with me one second. The membership and how that works.  
Me: I I've so I've been making notes, but at the moment, nothing.  
Them: Do you have any questions for me at all?  
Me: Sorry. Didn't help me.  
Them: Mhmm.  
Me: Okay. So I'm just gonna share with you  
Them: Sorry. If I'm going too fast, please, to tell me. Okay. So I'm just gonna share my second screen with you now.  
Me: I'm testing. The 2 left Say hello? Brilliant.  
Them: Fantastic.  
Me: Say, look at it.  
Them: Do let me know if you can see the screen, please.  
Me: It's free. To join. And  
Them: Brilliant. So  
Me: I'm just gonna show you  
Them: like I said, membership is free.  
Me: Proposition. How does  
Them: To join. And  
Me: Each  
Them: I'm just gonna show you, for example, what the mentoring and the coaching  
Me: Mint will give you Of  
Them: proposition, how that looks like. So  
Me: 8  
Them: Each mental will give you a brief background. Of what industry they specialize in. So for example,  
Me: Now If he I should  
Them: I'll show you right now how that work.  
Me: was it 68.  
Them: Now If you can see so each will give you a short bio.  
Me: Awesome. This time,  
Them: Whether they've got industry specific and  
Me: And, you know,  
Them: what support they can offer, whether it's climate, whether you're looking for funding,  
Me: Is that something that  
Them: general business support. So really and truly, it's  
Me: Connect So I just need to get a  
Them: just taking time to go through it. And if there's something that  
Me: Great. And then  
Them: might think had on our lives to connect with them so just literally request it free.  
Me: Okay.  
Them: And then they'll be able to they'll return your request within the day within 2 days.  
Me: 1 minute. You may That's  
Them: And then whatever time suits you. And you can have more than  
Me: looking at marketplace is  
Them: one mentor. So you may decide that, no. I may need help with  
Me: Page that could  
Them: you know, value proposition or, you know, looking at marketplaces, looking at pitch deck support is entirely up  
Me: want support there. So it's And  
Them: to you. It's whatever you want to what you want support with. They're there to help you.  
Me: And And they requested So that's it. You choose  
Them: So it's essentially just literally going on there and clicking on a mentoring  
Me: And then  
Them: then requesting So that's it. You choose your suitable mentor.  
Me: So that's fine.  
Them: You complete a booking request, and then you arrange a 1 to 1 session  
Me: So c h.  
Them: within 2 working days. So that's free.  
Me: Which  
Them: 4, for you. So programs  
Me: can't  
Them: at the moment so we have it's really interesting because  
Me: spoke for the.  
Them: we're just starting the year. We are launching a few of our  
Me: Set.  
Them: So for example, at the moment, we've got the product growth  
Me: It's  
Them: program. Online. So it it essentially tells you about  
Me: And it  
Them: startup with digital products and and just to see how  
Me: So Once again, I just find that people have that flex.  
Them: your product is for the market you're going in. If there's any boxes  
Me: Yes, sir.  
Them: or things that you haven't really thought of. So  
Me: So it tells you  
Them: once again, let me just find out a bit more of how that works. Yep. So  
Me: No. No. Okay. Yep.  
Them: so we tell you  
Me: So ready?  
Them: So you have not have you raised any fans at all?  
Me: Okay.  
Them: Okay. Yeah. So for people who have not raised more than 500,  
Me: That should be coming up soon.  
Them: thousand. So, yeah, we also have  
Me: 7 And let's  
Them: we also have product builder.  
Me: k.  
Them: That should be coming up soon. Once again, is is we've got so many different  
Me: Oh, yeah.  
Them: programs, and I'd say there's not one size fits all. So it depends on the business  
Me: You sent us  
Them: what they need, what the support you require.  
Me: We If you pick coming at see  
Them: And then, you know, you you can just sign up for these programs  
Me: Yep. And then you  
Them: and it's free. And, literally, if you click the  
Me: So  
Them: become a member, so you just select the virtual membership.  
Me: So that you just find the  
Them: And then you can sign up. So with the industry communities as well,  
Me: AI diverge  
Them: So you just sign up.  
Me: I need to also sign up today at  
Them: With the industry community, for example, if we go on AI and emerging tech,  
Me: that's something you like to come, and it's not been trained. But it's settled Tuesday.  
Them: You can join the AI community, and you can also sign  
Me: Time. And once it gets really  
Them: up for the AI breakfast event. Is that something you like to come? And it's normally free. Like, it's 2nd Tuesday of each month.  
Me: Just drink  
Them: And once it gets really interesting, we get different speakers within the AI.  
Me: Just waiting  
Them: Industry who would give us updates about markets and all sorts of interesting  
Me: I checked the box. Last one last 3 days. Inside outside. Once again, just  
Them: themes that's going on. So it's really I've attended I attended last  
Me: Join it. Xfinity. Let me get one 8  
Them: one last month. It was really insightful. So once again, you just  
Me: Okay. Really, really cool. Yeah. Do you have any  
Them: join the community, and you get all the information around  
Me: Hello?  
Them: AI and stuff like that.  
Me: No. No. No more questions. Fantastic.  
Them: Okay? Do you have any questions at all?  
Me: Yes. At the moment,  
Them: Fantastic.  
Me: Well, it's with NatWest. NatWest.  
Them: Yeah. So, essentially, that's that. At the moment, who's your business banking with as  
Me: I just If you may hold, support.  
Them: Well, not much. Okay. That's  
Me: Found that he was planning to raise 250 k.  
Them: fine. That's fine. So, I'll just give you a bit more. We also support  
Me: Of private How the words This We  
Them: founders who have managed to raise 250 k  
Me: Hi. How that works? You know, specialized relationship  
Them: of private investment or whatever, any form of investment,  
Me: Please  
Them: How that works is we will go under our  
Me: Yeah. Foreign  
Them: high growth portfolio. How that works is you have specialized relationship  
Me: If you need a foreign account, US euro account. And you also have a  
Them: relationship managed. You get,  
Me: 1 e three. So  
Them: foreign  
Me: come on the line if you want.  
Them: you need a foreign account, whether it's US or  
Me: Don't  
Them: euro account. You also have access to  
Me: We have don't And because  
Them: 1 year free banking. So whoever down the line, if you are looking to switch or anything like that, we offer bespoke banking for founders.  
Me: Let We make  
Them: And because you've raised that 250, it's classified as high growth.  
Me: k.  
Them: So, potentially, down the line, you may need extra lending or whatever.  
Me: But Nope. And do not  
Them: We may be able to look it up for you because you are a high growth business if you've managed to raise the 2.50 k of investment. But the proposition itself  
Me: Cut.  
Them: you do not need to bank with us. There's no catch. So  
Me: Enjoy the  
Them: my light has gone off in the office. I'm in I'm in a branch. So there's no,  
Me: So that's to your business. Thank you Okay. Amazing. Yeah. I'm excited to,  
Them: catch. There's nothing behind it. It's essentially just signing up and joining whichever community that you feel that is relevant  
Me: dive in and and take a look at these, the resources on  
Them: to your business and the support that you need.  
Me: yeah. Fine. Very happy to look through and to and to engage.  
Them: Fantastic.  
Me: Yes.  
Them: Fantastic. Brilliant. Brilliant. Fantastic. Yeah. So,  
Me: Now  
Them: essentially, that's what the Eagle Labs is about.  
Me: And you You know,  
Them: I'd also like to keep you on my network contact So if I'm doing any events around or if I come across anybody within  
Me: Connect I  
Them: you know, the industry that you are in and I feel like, okay. You guys can make a great connection. I'm more than happy  
Me: Looks it Next  
Them: to connect you and meet you in person. One of our events that hopefully, I'll be doing events  
Me: Oh, Amazing. Well, look, once again, thanks for reaching out. I I think  
Them: in the next couple of weeks months ahead. So, definitely, I just wanted to  
Me: There's there's certainly a lot of avenues to  
Them: meet you and just say hi and see how we can support.  
Me: Yeah. Get feedback and to grow an idea.  
Them: That's okay.  
Me: So I think you're ready to take the steps, which which I definitely am.  
Them: Yeah.  
Me: Brilliant. I'm very happy to to engage in the resources. And and, yeah, very happy to keep in touch.  
Them: Yeah. Brilliant.  
Me: If there are events and stuff,  
Them: Brilliant.  
Me: Okay. Happy to  
Them: Brilliant.  
Me: You get It  
Them: Fantastic. Fantastic. And I'll also encourage you  
Me: Is  
Them: to join the AI Breakfast event because it's really you get investors there. You get you get  
Me: So Yep. Fantastic.  
Them: so many different stakeholders there. So just really just sign up and just  
Me: Oh, Segment. Enjoy.  
Them: have a look come in and see how it's like.  
Me: It.  
Them: So I'll send you the information around that as well.  
Me: Well, let's  
Them: Okay. Oh, brilliant. Oh, thank you so much.  
Me: Yeah. You know, took Cheers.  
Them: Enjoy the rest of your day, and have a wonderful weekend.  
Me: And now  
Them: But let's catch up. Let me know when you want us to catch up again.  
Me: Oh,  
Them: And I don't know if you want us to catch up in 2 months time, 3 months  
Me: It is starting. See what Alright.  
Them: time. You let me know, and I'll make I'll I'll meet you, or we can just jump on  
Me: Yeah. Fantastic. Just curious, what's the, what what would you say has been a very  
Them: Teams if whatever is easier for you. You can just catch up to see where you are and if you need any further  
Me: interesting startup founder that that you've spoken to in the last  
Them: Boom.  
Me: in in the last few months. So Somebody Help. So It's It was was 8.  
Them: So for me,  
Me: Thank you.  
Them: and I I it was to do with health. So,  
Me: Can't Actually, the  
Them: it it was, it was a natural lady, and it was like,  
Me: it was around the female Yeah. The  
Them: people that have had cancer of some sort. I think it was more actually, yeah,  
Me: cancer diagnosis that someone has  
Them: it was around the female. Like, the the men initially, she started off with a cancer, looking at the cancer diagnosis that someone has  
Me: So Yes. It's I  
Them: had a cancer before  
Me: Myself  
Them: the chances and stuff like that. And now she's actually gone into female health  
Me: I'll get Pete to do an assessment to say that, okay. I'm at this age.  
Them: So there's a there's an AI thing that has a personalized so for example,  
Me: And he had this set tips that he had. If he had if he have the if he have the  
Them: myself of certain age, it'll get me to do an assessment  
Me: a plan for you to say, okay. You might  
Them: to say that, okay. I'm at this age.  
Me: see a doctor.  
Them: Have you had these symptoms? Have you had that? Have you had that? If you haven't or if you had, then what it does is it will draw out a plan for you to say, okay. You might need to see your doctor.  
Me: And she, like, s like  
Them: Because you may have menopause coming or no. You are in  
Me: Okay.  
Them: premenopausal kind of thing. So the AI will take all this information  
Me: Or, actually, when it  
Them: and tailor map it to each individual like a personalized,  
Me: a professional  
Them: prognosis to say, okay. You may need to see a doctor now.  
Me: as to why we're trying  
Them: Or, actually, you may need to x y zed. You may need to take vitamins and stuff like that.  
Me: Hi. Yep. And free  
Them: To see somebody. See a professional, first of all, and which I thought it blew me away.  
Me: You know, And in the  
Them: And one thing also is learning about AI. It's like,  
Me: things for me. The most interesting part of the health  
Them: everyday I learn something new or someone's doing I was doing this and you're like, wow.  
Me: And they AI me take You know,  
Them: You know? And it's essentially like connecting. I think for me,  
Me: the  
Them: the most interesting one I look at health and the impact of  
Me: I do.  
Them: what AI is doing to improve, you know, the  
Me: Wow. And Right.  
Them: the the for the user experience and also the efficiency  
Me: And prepaid. And I  
Them: And so I'm really intrigued every time I'm meeting people that especially around health and, like,  
Me: Actually, So yeah.  
Them: Wow.  
Me: That's  
Them: And every day, I'm surprised. Every day, AI is doing something amazing that, you know, you  
Me: You know,  
Them: think, wow. How'd you think about that? How'd you get this idea? You know?  
Me: Did I That's Yeah. I mean, there there is it's just there's just so many different applications for this.  
Them: So yeah. So that that that's one that's been one of the most fascinating things. And I  
Me: Yeah.  
Them: I'm still learning, you know, just to see, wow. Didn't know you can do that.  
Me: Emmet is is our creativity. And so, like, it's still taking time for people to  
Them: So yeah. Yeah.  
Me: you know, sort of Yeah. Oh, we can use this for this and then use that for this. Yeah. Yeah. So it is very interesting, and I'm  
Them: Yeah. Yes.  
Me: Yeah. Yes.  
Them: Yeah.  
Me: What's it?  
Them: Yeah.  
Me: So  
Them: Yeah. Yes.  
Me: Hi, ma'am.  
Them: Brilliant. Well, thank you so much.  
Me: Just  
Them: So, definitely, we'll send you all the information. I'll send you the ebill love deck.  
Me: Yeah. Very happy to. Thank you so much. Thank you.  
Them: I also send you the links below, so just click on it and register.  
Me: Alright. Thank you. Bye bye.  
Them: Just reach out if you need any further support at all. Okay. You're welcome. You're welcome. Have a wonderful day. Take care. Take care. Bye. 
"""
user_notes = """
- eagle labs
- monthly breakfast about AI in shoreditch
- demo directory/ free platform for founders and investors to connect
- corporate connections / a corporation will put a challenge on that platform, a founder get register
  - challenges ?
- accelerator pgroammes
  - plexal
  - lab locations -
- scaleups - they can support internationalisations
- Demo directory
  - investor directory, of angels and VCs
  - create a profile with a video
  - average funds raised is £1.2m. range is £20k - £35m
- corporate connections
- sign up virtual membership, you can sign up to communities.
- sign up online
  - review mentors profiles, you can pitch more than one mentor for other reasons.
- Product growth programme on eagle labs
- Virtual membership -
- industries - these are the communities
"""
prompt = f"""
<instructions>
Review the meeting_transcript, and the user_notes
User notes included in the summary must be highlighted with a "* *" syntax to show that this is the user's notes.
</instructions>
<meeting_transcript>
{meeting_transcript}
</meeting_transcript>
<user_notes>
{user_notes}
</user_notes>
"""

def text_response():  # renamed from text_stream since it's no longer streaming
    message = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=8192,
        temperature=0.3,
        messages=[{
            "role": "user",
            "content": prompt
        }],
        system=sys_prompt
    )
    # Print the actual content instead of the full message object
    print(message.content[0].text)
    return None

text_response()

Here's a formal summary of the meeting:

# Introduction
- Meeting between a founder and Eagle Labs representative
- Discussion about Eagle Labs services and support for startups/founders

# Eagle Labs Overview
- Part of Barclays Business Banking
- Established since 2015
- 42 lab locations across the country
- *38,000 event attendees supported*
- *17,000 businesses supported*
- Free membership for founders and startups

# Key Services

## Mentoring & Coaching
- Free mentoring and coaching services
- Multiple mentors available for different needs
- *Can request multiple mentors for different aspects of business*
- Response within 2 working days

## Networking Events
- *Monthly AI breakfast events in Shoreditch*
- Industry-specific events and workshops
- Annual demo days for investor meetings

## Investment Platform
- Demo Directory - free platform connecting founders and investors
- *400+ investors including angels and VCs*
- *Investment range: £20k - £25m*
- *Average fundraise: £1.2m*

